In [208]:
import pandas as pd
import os
from IPython.display import display
from pandasql import sqldf
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
dataset_path = 'dataset'
pysqldf = lambda q: sqldf(q, globals())


gold_df = pd.read_parquet(os.path.join(dataset_path, 'gold_partecipants.parquet'))
gold_df['population_display'] = gold_df['Population'].map('{:,}'.format)



In [209]:
def get_display_customdata(df):
    display = '<extra>'+ df.alpha_3 + '</extra><b>' + df.iso_names + ' (' + df.Year.astype(str) + ')</b><br><br>Medals:' + \
                         '<br>Gold: ' + df.Gold.astype(str) + \
                        '<br>Silver: ' + df.Silver.astype(str) + \
                        '<br>Bronze: ' + df.Bronze.astype(str) + \
                        '<br><br>Total medals collected: ' + df.TotalMedals.astype(str) + \
                        '<br>Partecipated to ' + \
                         df.events_partecipations_no.astype(str) + \
                        ' events with ' + df.distinct_athletes_no.astype(str) + ' athletes' + \
                        '<br><br>Country population: ' + df.population_display.astype(str)
    return display


In [210]:
gold_df = gold_df.assign(display= get_display_customdata(gold_df))
display(gold_df.head(3))



,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
0,AFG,1964,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,9744772,"9,744,772",<extra>AFG</extra><b>Afghanistan (1964)</b><br...
1,AFG,1968,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan,10637064,"10,637,064",<extra>AFG</extra><b>Afghanistan (1968)</b><br...
2,AFG,1972,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,11791222,"11,791,222",<extra>AFG</extra><b>Afghanistan (1972)</b><br...


In [211]:
from scipy import interpolate

def get_interpolation_coefficients(x, matrix_y, columns_names):
    n_cols = matrix_y.shape[1]
    interp = []
    for i in range(n_cols):
        interp.append(
            interpolate.splrep(x, matrix_y[:, i], s=0)
        )
    return interp

def get_derivatives(df, parameter='TotalMedals'):
    year_pivot_df = pd.pivot(df, index='Year', columns='NOC', values=parameter)
    year_pivot_df = year_pivot_df.fillna(value=0)
    velocity_df = pd.DataFrame(np.gradient(year_pivot_df.to_numpy(), 4, axis=0), columns=year_pivot_df.columns)
    velocity_df.set_index(year_pivot_df.index, inplace=True)
    vel_interpolations = get_interpolation_coefficients(velocity_df.index, velocity_df.to_numpy(), velocity_df.columns)

    acceleration_df = pd.DataFrame(np.gradient(velocity_df.to_numpy(), 4, axis=0), columns=year_pivot_df.columns)
    acceleration_df.set_index(year_pivot_df.index, inplace=True)
    acc_interpolations = get_interpolation_coefficients(acceleration_df.index, acceleration_df.to_numpy(), acceleration_df.columns)
    return velocity_df, acceleration_df



In [212]:
def add_worldwide_derivatives_to_df(df):
    vel, acc = get_derivatives(df)
    vel = vel.reset_index()
    acc = acc.reset_index()
    vel_melt = pd.melt(vel, id_vars='Year', var_name='NOC', value_name='velocity')
    acc_melt = pd.melt(acc, id_vars='Year', var_name='NOC', value_name='acceleration')
    gold_derivatives_df = df.merge(vel_melt, on=['NOC','Year'], how='left')
    gold_derivatives_df = gold_derivatives_df.merge(acc_melt, on=['NOC','Year'], how='left')
    return gold_derivatives_df

x_plot = np.linspace(1992, 2016, 50)

my_years = [2016, 2012, 2008]
gold_countries_derivatives_df = add_worldwide_derivatives_to_df(gold_df)
gold_countries_derivatives_df.to_parquet(os.path.join(dataset_path, 'gold_countries_derivatives.parquet'))

In [213]:
# Get year 2016
def get_recent_three_years(df):
    years = df.Year.unique()
    years = np.sort(years)
    years = years[-3:]
    return np.flip(years)

recent_years = get_recent_three_years(gold_df)
print(recent_years)

[2016 2012 2008]


In [214]:
def filter_by_year_my_country(df, my_country, my_year, threshold=0.2):
    # Get population of my country
    pop = df['Population'].loc[(df.NOC == my_country) & (df.Year== my_year)].iloc[0]

    # Define cut-off percentage threshold
    # pop_max = pop +- threshold%
    # threshold = 0.15 means pop +- 15%

    filtered_countries = df[df.Year == my_year]
    filtered_countries = filtered_countries[filtered_countries.Population.between(pop - pop*threshold, pop + pop*threshold )]
    filtered_countries = filtered_countries.sort_values(by='TotalMedals')
    display(filtered_countries)
    return filtered_countries

In [215]:
def get_choropleth_title_starts_with(sorting_parameter):
    title_starts_with = ''
    if sorting_parameter == 'TotalMedals':
        title_starts_with = 'Total medals collected'
    elif sorting_parameter in ['Gold', 'Silver', 'Bronze']:
        title_starts_with = f'{sorting_parameter} medals collected'
    elif sorting_parameter == 'velocity':
        title_starts_with = f'Best trend'
    elif sorting_parameter == 'acceleration':
        title_starts_with = f'Fastest improving'
    elif sorting_parameter == 'AthletePerEventPartecipation':
        title_starts_with = f'Athletes per event'
    elif sorting_parameter == 'AthletePerMedal':
        title_starts_with = f'Athletes per medal'
    elif sorting_parameter == 'EventPartecipationPerMedal':
        title_starts_with = f'Event partecipations per medal'
    elif sorting_parameter == 'PopulationPerMedal_thousands':
        title_starts_with = 'Population (thousands) per medal'
    return title_starts_with

def get_choropleth_colormap_title(sorting_parameter):
    colormap_title = ''
    if sorting_parameter == 'TotalMedals':
        colormap_title = 'Total medals'
    elif sorting_parameter in ['Gold', 'Silver', 'Bronze' ]:
        colormap_title = f'{sorting_parameter}'
    elif sorting_parameter == 'velocity':
        colormap_title = f'Velocity'
    elif sorting_parameter == 'acceleration':
        colormap_title = f'Acceleration'
    elif sorting_parameter == 'AthletePerEventPartecipation':
        colormap_title = f'Athl/Event'
    elif sorting_parameter == 'AthletePerMedal':
        colormap_title = f'Athl/Medal'
    elif sorting_parameter == 'EventPartecipationPerMedal':
        colormap_title = f'Event/medal'
    elif sorting_parameter == 'PopulationPerMedal_thousands':
        colormap_title = 'Pop/Medal'
    return colormap_title

def get_choropleth_colormap(sorting_parameter):
    colormap = "Emrld"
    if sorting_parameter in ['Gold', 'Silver', 'Bronze', 'TotalMedals' ]:
        colormap = "Emrld"
    elif sorting_parameter in ['velocity', 'acceleration']:
        colormap = 'RdYlGn'
    return colormap

def get_fig_choropleth_similar_size(df, my_country, show_years, sort_by = 'TotalMedals', threshold=0.2):
    # Show only my country and similar sized ones
    fig_cho_size = go.Figure()
    
    first = True
    for year in show_years:
        starting_data = filter_by_year_my_country(df, my_country, year, threshold)
        display(starting_data)

        visible = False
        if first:
            visible = True
            first = False
    
        fig_cho_size.add_trace( \
            go.Choropleth(
            locations=starting_data.alpha_3,
            z=starting_data[sort_by],
            colorscale=get_choropleth_colormap(sort_by),
            customdata=starting_data.display,
            hovertemplate='%{customdata}',
            text=starting_data.iso_names,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title = get_choropleth_colormap_title(sort_by),
            visible=visible
            )
        )


    title_starts_with = get_choropleth_title_starts_with(sort_by)
    y_lab_height = 0.98
    fig_cho_size.update_layout(
        title={
            'text': f'{title_starts_with} in year {show_years[0]} by country<br>'
                    f'Countries with population similar to '
                    f'{starting_data["iso_names"].loc[starting_data.alpha_3 == selected_country].iloc[0]} '
                    f'(&plusmn; 20%)',
            'y':1,
            'x':0.45,
            'xanchor': 'center',
            'yanchor': 'bottom',
            'xref': 'paper',
            'yref': 'paper',
            'pad': {"b": 40, "t": 10},
        },
    
           # list([{'label': i,
           #                     'args': [
           #                          {
           #                          'y':[df_select[i]],
           #                          }],
           #                     'method': 'restyle'} for i in df_select.columns[1:]]),
        updatemenus=[
            dict(
                active=0,
                buttons=list([
                    dict(label=str(show_years[0]),
                         method="update",
                         args=[{"visible": [True, False, False]},
                               {"title": f'{title_starts_with} in year {show_years[0]} by country<br>'
                                f'Countries with population similar to '
                                f'{starting_data["iso_names"].loc[starting_data.alpha_3 == selected_country].iloc[0]} '
                                f'(&plusmn; 20%)',}]),
                    dict(label=str(show_years[1]),
                         method="update",
                         args=[{"visible": [False, True, False]},
                               {"title": f'{title_starts_with} in year {show_years[1]} by country<br>'
                                f'Countries with population similar to '
                                f'{starting_data["iso_names"].loc[starting_data.alpha_3 == selected_country].iloc[0]} '
                                f'(&plusmn; 20%)',}]),
                    dict(label=str(show_years[2]),
                         method="update",
                         args=[{"visible": [False, False, True]},
                               {"title": f'{title_starts_with} in year {show_years[2]} by country<br>'
                                f'Countries with population similar to '
                                f'{starting_data["iso_names"].loc[starting_data.alpha_3 == selected_country].iloc[0]} '
                                f'(&plusmn; 20%)',}]),
                ]),
                x = 0,
                xanchor = 'left',
                y = y_lab_height,
                yanchor = 'top',
                pad={"t": 10}
            )
        ],
        annotations=[
            dict(text="Select Olympic year:", x=0, xref="paper", y=1, yref="paper",
                                 align="left", showarrow=False)]
    
    )
    return fig_cho_size


In [216]:

import pycountry

selected_country = "ESP"

if selected_country in gold_df.alpha_3.unique():
    fig = get_fig_choropleth_similar_size(gold_df, selected_country, recent_years)
    fig.show(renderer='browser')
else:
    print(f"What did you mean by {selected_country}?")
    print(pycountry.countries.search_fuzzy(selected_country))


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1407,MYA,2016,8,7,0,0,8,0,8,0,Myanmar,MMR,Myanmar,53045199,"53,045,199",<extra>MMR</extra><b>Myanmar (2016)</b><br><br...
1839,SUD,2016,6,6,0,0,6,0,6,0,Sudan,SDN,Sudan,39847433,"39,847,433",<extra>SDN</extra><b>Sudan (2016)</b><br><br>M...
1920,TAN,2016,7,7,0,0,5,0,5,0,Tanzania,TZA,"Tanzania, United Republic of",53049231,"53,049,231","<extra>TZA</extra><b>Tanzania, United Republic..."
2052,UGA,2016,23,21,0,0,14,0,14,0,Uganda,UGA,Uganda,39649173,"39,649,173",<extra>UGA</extra><b>Uganda (2016)</b><br><br>...
41,ALG,2016,74,64,0,0,48,2,50,2,Algeria,DZA,Algeria,40551398,"40,551,398",<extra>DZA</extra><b>Algeria (2016)</b><br><br...
84,ARG,2016,232,215,0,3,77,1,81,4,Argentina,ARG,Argentina,43590368,"43,590,368",<extra>ARG</extra><b>Argentina (2016)</b><br><...
471,COL,2016,157,143,3,3,80,2,88,8,Colombia,COL,Colombia,48175048,"48,175,048",<extra>COL</extra><b>Colombia (2016)</b><br><b...
1619,POL,2016,289,234,6,2,138,3,149,11,Poland,POL,Poland,37970087,"37,970,087",<extra>POL</extra><b>Poland (2016)</b><br><br>...
2058,UKR,2016,272,204,4,2,137,5,148,11,Ukraine,UKR,Ukraine,45004673,"45,004,673",<extra>UKR</extra><b>Ukraine (2016)</b><br><br...
1079,KEN,2016,80,79,1,6,28,6,41,13,Kenya,KEN,Kenya,49051531,"49,051,531",<extra>KEN</extra><b>Kenya (2016)</b><br><br>M...


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1407,MYA,2016,8,7,0,0,8,0,8,0,Myanmar,MMR,Myanmar,53045199,"53,045,199",<extra>MMR</extra><b>Myanmar (2016)</b><br><br...
1839,SUD,2016,6,6,0,0,6,0,6,0,Sudan,SDN,Sudan,39847433,"39,847,433",<extra>SDN</extra><b>Sudan (2016)</b><br><br>M...
1920,TAN,2016,7,7,0,0,5,0,5,0,Tanzania,TZA,"Tanzania, United Republic of",53049231,"53,049,231","<extra>TZA</extra><b>Tanzania, United Republic..."
2052,UGA,2016,23,21,0,0,14,0,14,0,Uganda,UGA,Uganda,39649173,"39,649,173",<extra>UGA</extra><b>Uganda (2016)</b><br><br>...
41,ALG,2016,74,64,0,0,48,2,50,2,Algeria,DZA,Algeria,40551398,"40,551,398",<extra>DZA</extra><b>Algeria (2016)</b><br><br...
84,ARG,2016,232,215,0,3,77,1,81,4,Argentina,ARG,Argentina,43590368,"43,590,368",<extra>ARG</extra><b>Argentina (2016)</b><br><...
471,COL,2016,157,143,3,3,80,2,88,8,Colombia,COL,Colombia,48175048,"48,175,048",<extra>COL</extra><b>Colombia (2016)</b><br><b...
1619,POL,2016,289,234,6,2,138,3,149,11,Poland,POL,Poland,37970087,"37,970,087",<extra>POL</extra><b>Poland (2016)</b><br><br>...
2058,UKR,2016,272,204,4,2,137,5,148,11,Ukraine,UKR,Ukraine,45004673,"45,004,673",<extra>UKR</extra><b>Ukraine (2016)</b><br><br...
1079,KEN,2016,80,79,1,6,28,6,41,13,Kenya,KEN,Kenya,49051531,"49,051,531",<extra>KEN</extra><b>Kenya (2016)</b><br><br>M...


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1406,MYA,2012,6,6,0,0,6,0,6,0,Myanmar,MMR,Myanmar,51413703,"51,413,703",<extra>MMR</extra><b>Myanmar (2012)</b><br><br...
1919,TAN,2012,6,6,0,0,5,0,5,0,Tanzania,TZA,"Tanzania, United Republic of",47053033,"47,053,033","<extra>TZA</extra><b>Tanzania, United Republic..."
83,ARG,2012,148,137,2,1,63,1,67,4,Argentina,ARG,Argentina,41733271,"41,733,271",<extra>ARG</extra><b>Argentina (2012)</b><br><...
1687,RSA,2012,145,124,1,3,59,2,65,6,South Africa,ZAF,South Africa,52832659,"52,832,659",<extra>ZAF</extra><b>South Africa (2012)</b><b...
470,COL,2012,120,101,4,1,75,3,83,8,Colombia,COL,Colombia,46075721,"46,075,721",<extra>COL</extra><b>Colombia (2012)</b><br><b...
1618,POL,2012,256,210,6,2,141,2,151,10,Poland,POL,Poland,38063164,"38,063,164",<extra>POL</extra><b>Poland (2012)</b><br><br>...
1078,KEN,2012,51,47,5,2,23,4,34,11,Kenya,KEN,Kenya,44343469,"44,343,469",<extra>KEN</extra><b>Kenya (2012)</b><br><br>M...
629,ESP,2012,339,278,4,3,121,10,138,17,Spain,ESP,Spain,46773055,"46,773,055",<extra>ESP</extra><b>Spain (2012)</b><br><br>M...
2057,UKR,2012,304,230,9,6,159,5,179,20,Ukraine,UKR,Ukraine,45593342,"45,593,342",<extra>UKR</extra><b>Ukraine (2012)</b><br><br...
1101,KOR,2012,312,250,7,13,122,8,150,28,South Korea,KOR,"Korea, Republic of",50199853,"50,199,853","<extra>KOR</extra><b>Korea, Republic of (2012)..."


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1406,MYA,2012,6,6,0,0,6,0,6,0,Myanmar,MMR,Myanmar,51413703,"51,413,703",<extra>MMR</extra><b>Myanmar (2012)</b><br><br...
1919,TAN,2012,6,6,0,0,5,0,5,0,Tanzania,TZA,"Tanzania, United Republic of",47053033,"47,053,033","<extra>TZA</extra><b>Tanzania, United Republic..."
83,ARG,2012,148,137,2,1,63,1,67,4,Argentina,ARG,Argentina,41733271,"41,733,271",<extra>ARG</extra><b>Argentina (2012)</b><br><...
1687,RSA,2012,145,124,1,3,59,2,65,6,South Africa,ZAF,South Africa,52832659,"52,832,659",<extra>ZAF</extra><b>South Africa (2012)</b><b...
470,COL,2012,120,101,4,1,75,3,83,8,Colombia,COL,Colombia,46075721,"46,075,721",<extra>COL</extra><b>Colombia (2012)</b><br><b...
1618,POL,2012,256,210,6,2,141,2,151,10,Poland,POL,Poland,38063164,"38,063,164",<extra>POL</extra><b>Poland (2012)</b><br><br>...
1078,KEN,2012,51,47,5,2,23,4,34,11,Kenya,KEN,Kenya,44343469,"44,343,469",<extra>KEN</extra><b>Kenya (2012)</b><br><br>M...
629,ESP,2012,339,278,4,3,121,10,138,17,Spain,ESP,Spain,46773055,"46,773,055",<extra>ESP</extra><b>Spain (2012)</b><br><br>M...
2057,UKR,2012,304,230,9,6,159,5,179,20,Ukraine,UKR,Ukraine,45593342,"45,593,342",<extra>UKR</extra><b>Ukraine (2012)</b><br><br...
1101,KOR,2012,312,250,7,13,122,8,150,28,South Korea,KOR,"Korea, Republic of",50199853,"50,199,853","<extra>KOR</extra><b>Korea, Republic of (2012)..."


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1405,MYA,2008,7,6,0,0,7,0,7,0,Myanmar,MMR,Myanmar,49929642,"49,929,642",<extra>MMR</extra><b>Myanmar (2008)</b><br><br...
1918,TAN,2008,9,9,0,0,6,0,6,0,Tanzania,TZA,"Tanzania, United Republic of",41853944,"41,853,944","<extra>TZA</extra><b>Tanzania, United Republic..."
1686,RSA,2008,179,134,0,0,88,1,89,1,South Africa,ZAF,South Africa,49779472,"49,779,472",<extra>ZAF</extra><b>South Africa (2008)</b><b...
469,COL,2008,90,67,1,0,73,1,75,2,Colombia,COL,Colombia,44254972,"44,254,972",<extra>COL</extra><b>Colombia (2008)</b><br><b...
82,ARG,2008,145,132,4,2,65,0,71,6,Argentina,ARG,Argentina,40080159,"40,080,159",<extra>ARG</extra><b>Argentina (2008)</b><br><...
1617,POL,2008,316,257,1,3,157,6,167,10,Poland,POL,Poland,38125759,"38,125,759",<extra>POL</extra><b>Poland (2008)</b><br><br>...
1077,KEN,2008,47,46,5,6,23,4,38,15,Kenya,KEN,Kenya,39791984,"39,791,984",<extra>KEN</extra><b>Kenya (2008)</b><br><br>M...
628,ESP,2008,355,283,3,5,133,10,151,18,Spain,ESP,Spain,45954106,"45,954,106",<extra>ESP</extra><b>Spain (2008)</b><br><br>M...
2056,UKR,2008,327,243,15,7,161,5,188,27,Ukraine,UKR,Ukraine,46258189,"46,258,189",<extra>UKR</extra><b>Ukraine (2008)</b><br><br...
1100,KOR,2008,329,265,8,13,122,10,153,31,South Korea,KOR,"Korea, Republic of",49054708,"49,054,708","<extra>KOR</extra><b>Korea, Republic of (2008)..."


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,population_display,display
1405,MYA,2008,7,6,0,0,7,0,7,0,Myanmar,MMR,Myanmar,49929642,"49,929,642",<extra>MMR</extra><b>Myanmar (2008)</b><br><br...
1918,TAN,2008,9,9,0,0,6,0,6,0,Tanzania,TZA,"Tanzania, United Republic of",41853944,"41,853,944","<extra>TZA</extra><b>Tanzania, United Republic..."
1686,RSA,2008,179,134,0,0,88,1,89,1,South Africa,ZAF,South Africa,49779472,"49,779,472",<extra>ZAF</extra><b>South Africa (2008)</b><b...
469,COL,2008,90,67,1,0,73,1,75,2,Colombia,COL,Colombia,44254972,"44,254,972",<extra>COL</extra><b>Colombia (2008)</b><br><b...
82,ARG,2008,145,132,4,2,65,0,71,6,Argentina,ARG,Argentina,40080159,"40,080,159",<extra>ARG</extra><b>Argentina (2008)</b><br><...
1617,POL,2008,316,257,1,3,157,6,167,10,Poland,POL,Poland,38125759,"38,125,759",<extra>POL</extra><b>Poland (2008)</b><br><br>...
1077,KEN,2008,47,46,5,6,23,4,38,15,Kenya,KEN,Kenya,39791984,"39,791,984",<extra>KEN</extra><b>Kenya (2008)</b><br><br>M...
628,ESP,2008,355,283,3,5,133,10,151,18,Spain,ESP,Spain,45954106,"45,954,106",<extra>ESP</extra><b>Spain (2008)</b><br><br>M...
2056,UKR,2008,327,243,15,7,161,5,188,27,Ukraine,UKR,Ukraine,46258189,"46,258,189",<extra>UKR</extra><b>Ukraine (2008)</b><br><br...
1100,KOR,2008,329,265,8,13,122,10,153,31,South Korea,KOR,"Korea, Republic of",49054708,"49,054,708","<extra>KOR</extra><b>Korea, Republic of (2008)..."


In [217]:
# World View

def get_fig_choropleth_world(df, show_years, sort_by = 'TotalMedals'):
    fig_world = go.Figure()

    first = True

    for year in show_years:
        year_df = df.loc[df.Year == year]
        dupes = year_df.loc[year_df.duplicated(subset=['Year', 'alpha_3'], keep=False)]

        visible = False
        if first:
            visible = True
            first = False

        fig_world.add_trace( \
            go.Choropleth(
            locations=year_df.alpha_3,
            z=year_df[sort_by],
            colorscale=get_choropleth_colormap(sort_by),
            customdata=year_df.display,
            hovertemplate='%{customdata}',
            text=year_df.iso_names,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title = get_choropleth_colormap_title(sort_by),
            visible=visible,
            )
        )

    title_starts_with = get_choropleth_title_starts_with(sort_by)
    y_label_height = 0.98
    fig_world.update_layout(
        title={
            'text': f'{title_starts_with} in year {show_years[0]} worldwide',
            'y':1,
            'x':0.45,
            'xanchor': 'center',
            'yanchor': 'bottom',
            'xref': 'paper',
            'yref': 'paper',
            'pad': {"b": 20, "t": 10},
        },
        updatemenus=[
            dict(
                active=0,
                buttons=list([
                    dict(label=str(show_years[0]),
                         method="update",
                         args=[{"visible": [True, False, False]},
                               {"title": f'{title_starts_with} in year {show_years[0]} worldwide'}]),
                    dict(label=str(show_years[1]),
                         method="update",
                         args=[{"visible": [False, True, False]},
                               {"title": f'{title_starts_with} in year {show_years[1]} worldwide'}]),
                    dict(label=str(show_years[2]),
                         method="update",
                         args=[{"visible": [False, False, True]},
                               {"title": f'{title_starts_with} in year {show_years[2]} worldwide'}]),
                ]),
                x = 0,
                xanchor = 'left',
                y = y_label_height,
                yanchor = 'top',
                pad={"t": 10}
            )
        ],
        annotations=[
            dict(text="Select Olympic year:", x=0, xref="paper", y=1, yref="paper",
                                 align="left", showarrow=False)]

    )
    return fig_world


In [218]:
fig = get_fig_choropleth_world(gold_df, recent_years)
fig.show(renderer='browser')

In [219]:
fig = get_fig_choropleth_world(gold_countries_derivatives_df, recent_years, 'velocity')
fig.show(renderer='browser')

In [220]:
fig = get_fig_choropleth_world(gold_countries_derivatives_df, recent_years, 'acceleration')
fig.show(renderer='browser')

In [221]:
gold_metrics_df = pd.read_parquet(os.path.join(dataset_path, 'gold_countries_year_metrics.parquet'))
gold_metrics_df['population_display'] = gold_metrics_df['Population'].map('{:,}'.format)
gold_metrics_df = gold_metrics_df.assign(display= get_display_customdata(gold_metrics_df))

display(gold_metrics_df.head(3))


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,PopulationPerMedal_thousands,EventPartecipationPerMedal,AthletePerMedal,AthletePerEventPartecipation,population_display,display
7,AFG,2008,4,4,1,0,3,0,4,1,Afghanistan,AFG,Afghanistan,27722281,27722.281,4.0,4.0,1.00,"27,722,281",<extra>AFG</extra><b>Afghanistan (2008)</b><br...
8,AFG,2012,6,6,1,0,5,0,6,1,Afghanistan,AFG,Afghanistan,31161378,31161.378,6.0,6.0,1.00,"31,161,378",<extra>AFG</extra><b>Afghanistan (2012)</b><br...
15,AHO,1988,4,3,0,0,3,1,4,1,Curacao,CUW,Curaçao,149254,149.254,4.0,3.0,0.75,"149,254",<extra>CUW</extra><b>Curaçao (1988)</b><br><br...


In [222]:
fig = get_fig_choropleth_world(gold_metrics_df, recent_years, 'AthletePerEventPartecipation')
fig.show(renderer='browser')



In [223]:
fig = get_fig_choropleth_world(gold_metrics_df, recent_years, 'PopulationPerMedal_thousands')
fig.show(renderer='browser')

In [224]:
fig = get_fig_choropleth_world(gold_metrics_df, recent_years, 'AthletePerMedal')
fig.show(renderer='browser')
